# Rentas personales y de los hogares por secciones

En este cuaderno tratamos los datos del INE sobre las rentas netas por persona y por hogar en 2015 y 2017, distribuidas por secciones electorales. El INE declara que se trata de una estadística experimental, y sin duda se nota, porque el formato nos parece algo complicado, y existen bastantes datos que no aparecen.

Reconocemos que el código utilizado puede haber sido algo repetitivo y poco 'pythonista', pero pensamos que era necesario para ir teniendo una idea clara de los resultados que íbamos obteniendo paso a paso.

El fichero excel original se trato muy ligeramente, eliminado las filas iniciales, que incluian encabezamientos que ocupaban varias filas fusionadas.

Comenzamos con la importación de las librerias habituales y el fichero excel.

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import boto3

BUCKET_NAME = 'electomedia' 

# sustituir por credenciales de acceso. 
s3 = boto3.resource('s3', aws_access_key_id = 'XXXXXXXX', 
                          aws_secret_access_key= 'XXXXXXXXXXXXXX')

In [4]:
import botocore.exceptions

KEY = 'INE/Renta-Secciones-Elect.xlsx' 

try:
    s3.Bucket(BUCKET_NAME).download_file(KEY, 'Renta-Secciones-Elect.xlsx')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [5]:
rentas = pd.read_excel('Renta-Secciones-Elect.xlsx')

Desde el principio nos damos cuenta que la identificación del territorio va a ser complicada, ya que los registros incluyen a secciones, distritos y municipios. 

Hay municipios, como el primero de ellos, Alegría-Dulantzi, del que no hay datos; otros como Durango solo tiene datos del municipio.

Las columnas son de rentas netas o brutas medias (RNM o RBM) por persona u hogar (per o hog), o también mediana (Med) de renta de unidad de consumo (UC) para distintos años.   

In [6]:
rentas.head()

,Territorio,RNM per 2017,RNM per 2016,RNM per 2015,RNM hog 2017,RNM hog 2016,RNM hog 2015,Med R UC 2017,Med R UC 2016,Med R UC 2015,RBM per 2017,RBM per 2016,RBM per 2015,RBM hog 2017,RBM hog 2016,RBM hog 2015
0,01001 Alegría-Dulantzi,,,,,,,,,,,,,,,
1,0100101 Alegría-Dulantzi distrito 01,,,,,,,,,,,,,,,
2,0100101001 Alegría-Dulantzi sección 01001,,,,,,,,,,,,,,,
3,0100101002 Alegría-Dulantzi sección 01002,,,,,,,,,,,,,,,
4,01002 Amurrio,13862,13691,13800,34411,33936,34421,,,,,,,,,


In [7]:
rentas.shape

(55087, 16)

En principio, si tenemos datos de municipio y no de las secciones, rellenaremos estas con el municipio, como ocurre con el municipio de Amurrio, por ejemplo; si no tenemos nada, utilizaremos la media de toda la columna.

Comenzamos por tratar la columna del territorio, transformándola en una lista dentro de la columna 'Código'.

In [8]:
rentas['Código'] = rentas['Territorio'].str.split(pat = ' ', n = 1)

In [9]:
rentas.head()

,Territorio,RNM per 2017,RNM per 2016,RNM per 2015,RNM hog 2017,RNM hog 2016,RNM hog 2015,Med R UC 2017,Med R UC 2016,Med R UC 2015,RBM per 2017,RBM per 2016,RBM per 2015,RBM hog 2017,RBM hog 2016,RBM hog 2015,Código
0,01001 Alegría-Dulantzi,,,,,,,,,,,,,,,,"[01001, Alegría-Dulantzi]"
1,0100101 Alegría-Dulantzi distrito 01,,,,,,,,,,,,,,,,"[0100101, Alegría-Dulantzi distrito 01]"
2,0100101001 Alegría-Dulantzi sección 01001,,,,,,,,,,,,,,,,"[0100101001, Alegría-Dulantzi sección 01001]"
3,0100101002 Alegría-Dulantzi sección 01002,,,,,,,,,,,,,,,,"[0100101002, Alegría-Dulantzi sección 01002]"
4,01002 Amurrio,13862,13691,13800,34411,33936,34421,,,,,,,,,,"[01002, Amurrio]"


In [10]:
rentas.shape

(55087, 17)

In [11]:
rentas['Código'][0][0]

'01001'

Extraemos el primer elemento numérico, que nos servirá para diferenciar entre munnicipio, distrito y sección, y lo almacenamos en la columna 'Código 1'. Vemos el ejemplo de Amurrio.

In [12]:
rentas['Código1'] = rentas['Código'].apply(lambda x : x[0])

In [13]:
rentas[4:10]

,Territorio,RNM per 2017,RNM per 2016,RNM per 2015,RNM hog 2017,RNM hog 2016,RNM hog 2015,Med R UC 2017,Med R UC 2016,Med R UC 2015,RBM per 2017,RBM per 2016,RBM per 2015,RBM hog 2017,RBM hog 2016,RBM hog 2015,Código,Código1
4,01002 Amurrio,13862,13691,13800,34411,33936,34421,,,,,,,,,,"[01002, Amurrio]",01002
5,0100201 Amurrio distrito 01,,,,,,,,,,,,,,,,"[0100201, Amurrio distrito 01]",0100201
6,0100201001 Amurrio sección 01001,,,,,,,,,,,,,,,,"[0100201001, Amurrio sección 01001]",0100201001
7,0100201002 Amurrio sección 01002,,,,,,,,,,,,,,,,"[0100201002, Amurrio sección 01002]",0100201002
8,0100201003 Amurrio sección 01003,,,,,,,,,,,,,,,,"[0100201003, Amurrio sección 01003]",0100201003
9,0100201004 Amurrio sección 01004,,,,,,,,,,,,,,,,"[0100201004, Amurrio sección 01004]",0100201004


Ojo que los valores que no aparecen son en muchos casos cadenas vacias, no NaN.

In [14]:
rentas.iloc[6,5]

' '

Sabemos que las cinco primeras cifras del código numérico definen el municipio, por lo que lo almacenamos en una columna, cod_aux.

In [15]:
rentas['cod_aux'] = rentas['Código1'].str[0:5]

In [16]:
rentas

,Territorio,RNM per 2017,RNM per 2016,RNM per 2015,RNM hog 2017,RNM hog 2016,RNM hog 2015,Med R UC 2017,Med R UC 2016,Med R UC 2015,RBM per 2017,RBM per 2016,RBM per 2015,RBM hog 2017,RBM hog 2016,RBM hog 2015,Código,Código1,cod_aux
0,01001 Alegría-Dulantzi,,,,,,,,,,,,,,,,"[01001, Alegría-Dulantzi]",01001,01001
1,0100101 Alegría-Dulantzi distrito 01,,,,,,,,,,,,,,,,"[0100101, Alegría-Dulantzi distrito 01]",0100101,01001
2,0100101001 Alegría-Dulantzi sección 01001,,,,,,,,,,,,,,,,"[0100101001, Alegría-Dulantzi sección 01001]",0100101001,01001
3,0100101002 Alegría-Dulantzi sección 01002,,,,,,,,,,,,,,,,"[0100101002, Alegría-Dulantzi sección 01002]",0100101002,01001
4,01002 Amurrio,13862,13691,13800,34411,33936,34421,,,,,,,,,,"[01002, Amurrio]",01002,01002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55082,5200108011 Melilla sección 08011,16433,16373,15847,66352,64892,62632,21350,22750,23450,18859,18795,18168,76148,74491,71808,"[5200108011, Melilla sección 08011]",5200108011,52001
55083,5200108012 Melilla sección 08012,17350,17185,16792,50730,50458,50839,23450,23450,22050,19910,19645,19280,58213,57680,58372,"[5200108012, Melilla sección 08012]",5200108012,52001
55084,5200108013 Melilla sección 08013,12553,12705,11823,37816,38720,36729,16450,16450,15750,14152,14324,13312,42633,43654,41356,"[5200108013, Melilla sección 08013]",5200108013,52001
55085,5200108014 Melilla sección 08014,8906,8872,8937,29898,30421,31384,12950,13650,13650,9651,9628,9705,32401,33013,34083,"[5200108014, Melilla sección 08014]",5200108014,52001


Vemos que en una de las columnas, y se supone que en todas, hay elementos como puntos, o espacios en blanco. Más problemas. 

In [17]:
rentas['RNM per 2017'].value_counts()

.        4916
          476
30210     166
3431       28
10966      24
         ... 
6479        1
6478        1
6476        1
22859       1
16383       1
Name: RNM per 2017, Length: 12055, dtype: int64

Podemos diferenciar los registros de los municipios porque su código contiene solo 5 caracteres. Los almacenamos en un df, rentasmuni, que nos será útil más adelante. A la colunma Código1 la llamamos también cod aux

In [18]:
rentas_muni = rentas.loc[rentas['Código1'].str.len() == 5]

In [19]:
rentas_muni

,Territorio,RNM per 2017,RNM per 2016,RNM per 2015,RNM hog 2017,RNM hog 2016,RNM hog 2015,Med R UC 2017,Med R UC 2016,Med R UC 2015,RBM per 2017,RBM per 2016,RBM per 2015,RBM hog 2017,RBM hog 2016,RBM hog 2015,Código,Código1,cod_aux
0,01001 Alegría-Dulantzi,,,,,,,,,,,,,,,,"[01001, Alegría-Dulantzi]",01001,01001
4,01002 Amurrio,13862,13691,13800,34411,33936,34421,,,,,,,,,,"[01002, Amurrio]",01002,01002
13,01003 Aramaio,16177,17194,15786,43021,46567,43199,,,,,,,,,,"[01003, Aramaio]",01003,01003
16,01004 Artziniega,12530,13185,12722,31490,32680,32269,,,,,,,,,,"[01004, Artziniega]",01004,01004
19,01006 Armiñón,13115,11330,13040,31165,28266,30699,,,,,,,,,,"[01006, Armiñón]",01006,01006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54961,50901 Biel,16414,17697,16613,25367,27274,26506,19950,20650,18550,20075,21276,20143,31024,32789,32139,"[50901, Biel]",50901,50901
54964,50902 Marracos,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,"[50902, Marracos]",50902,50902
54967,50903 Villamayor de Gállego,13018,12362,12136,34237,32882,32498,18550,17850,17150,15594,14789,14493,41015,39339,38810,"[50903, Villamayor de Gállego]",50903,50903
54971,51001 Ceuta,10767,10696,10696,35274,35381,35577,12950,12950,12950,12094,11988,12017,39622,39656,39971,"[51001, Ceuta]",51001,51001


In [20]:
rentas_muni['RBM per 2017'].value_counts()

.        1293
          530
10613       5
11426       5
11277       5
         ... 
12878       1
12877       1
12876       1
21067       1
8193        1
Name: RBM per 2017, Length: 4620, dtype: int64

In [21]:
#rentas_muni.rename(columns={'Código1' : 'cod_aux'})

Tendremos que hacer posteriormemnte un merge con el df rentas, así que conviene cambiar las columnas de nombre en rentas muni.

In [22]:
rentas_muni = rentas_muni.rename(columns={'RNM per 2017' : 'renta_per_2017', 'RNM per 2015' : 'renta_per_2015', 'RNM hog 2017': 'renta_hog_2017', 'RNM hog 2015': 'renta_hog_2015'})

In [23]:
rentas_muni

,Territorio,renta_per_2017,RNM per 2016,renta_per_2015,renta_hog_2017,RNM hog 2016,renta_hog_2015,Med R UC 2017,Med R UC 2016,Med R UC 2015,RBM per 2017,RBM per 2016,RBM per 2015,RBM hog 2017,RBM hog 2016,RBM hog 2015,Código,Código1,cod_aux
0,01001 Alegría-Dulantzi,,,,,,,,,,,,,,,,"[01001, Alegría-Dulantzi]",01001,01001
4,01002 Amurrio,13862,13691,13800,34411,33936,34421,,,,,,,,,,"[01002, Amurrio]",01002,01002
13,01003 Aramaio,16177,17194,15786,43021,46567,43199,,,,,,,,,,"[01003, Aramaio]",01003,01003
16,01004 Artziniega,12530,13185,12722,31490,32680,32269,,,,,,,,,,"[01004, Artziniega]",01004,01004
19,01006 Armiñón,13115,11330,13040,31165,28266,30699,,,,,,,,,,"[01006, Armiñón]",01006,01006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54961,50901 Biel,16414,17697,16613,25367,27274,26506,19950,20650,18550,20075,21276,20143,31024,32789,32139,"[50901, Biel]",50901,50901
54964,50902 Marracos,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,"[50902, Marracos]",50902,50902
54967,50903 Villamayor de Gállego,13018,12362,12136,34237,32882,32498,18550,17850,17150,15594,14789,14493,41015,39339,38810,"[50903, Villamayor de Gállego]",50903,50903
54971,51001 Ceuta,10767,10696,10696,35274,35381,35577,12950,12950,12950,12094,11988,12017,39622,39656,39971,"[51001, Ceuta]",51001,51001


In [24]:
rentas_muni.columns

Index(['Territorio', 'renta_per_2017', 'RNM per 2016', 'renta_per_2015',
       'renta_hog_2017', 'RNM hog 2016', 'renta_hog_2015', 'Med R UC 2017',
       'Med R UC 2016', 'Med R UC 2015', 'RBM per 2017', 'RBM per 2016',
       'RBM per 2015', 'RBM hog 2017', 'RBM hog 2016', 'RBM hog 2015',
       'Código', 'Código1', 'cod_aux'],
      dtype='object')

Nos quedamos con las columnas que más nos interesan de rentas muni, aparte del códido de identificación.

In [25]:
rentas_muni = rentas_muni[['cod_aux', 'renta_per_2017', 'renta_per_2015', 'renta_hog_2017',  'renta_hog_2015' ]]

In [26]:
rentas_muni

,cod_aux,renta_per_2017,renta_per_2015,renta_hog_2017,renta_hog_2015
0,01001,,,,
4,01002,13862,13800,34411,34421
13,01003,16177,15786,43021,43199
16,01004,12530,12722,31490,32269
19,01006,13115,13040,31165,30699
...,...,...,...,...,...
54961,50901,16414,16613,25367,26506
54964,50902,.,.,.,.
54967,50903,13018,12136,34237,32498
54971,51001,10767,10696,35274,35577


Ahora hacemos el merge con el df original, mediante la columna que identifica el municipio en ambas. Así podremos rellenar las columnas de las secciones en aquellos municipios en los que conozcamos sus datos a nivel agragado.

In [28]:
rentas_result = pd.merge(rentas, rentas_muni, how="outer", on='cod_aux')

In [29]:
rentas_result

,Territorio,RNM per 2017,RNM per 2016,RNM per 2015,RNM hog 2017,RNM hog 2016,RNM hog 2015,Med R UC 2017,Med R UC 2016,Med R UC 2015,...,RBM hog 2017,RBM hog 2016,RBM hog 2015,Código,Código1,cod_aux,renta_per_2017,renta_per_2015,renta_hog_2017,renta_hog_2015
0,01001 Alegría-Dulantzi,,,,,,,,,,...,,,,"[01001, Alegría-Dulantzi]",01001,01001,,,,
1,0100101 Alegría-Dulantzi distrito 01,,,,,,,,,,...,,,,"[0100101, Alegría-Dulantzi distrito 01]",0100101,01001,,,,
2,0100101001 Alegría-Dulantzi sección 01001,,,,,,,,,,...,,,,"[0100101001, Alegría-Dulantzi sección 01001]",0100101001,01001,,,,
3,0100101002 Alegría-Dulantzi sección 01002,,,,,,,,,,...,,,,"[0100101002, Alegría-Dulantzi sección 01002]",0100101002,01001,,,,
4,01002 Amurrio,13862,13691,13800,34411,33936,34421,,,,...,,,,"[01002, Amurrio]",01002,01002,13862,13800,34411,34421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55082,5200108011 Melilla sección 08011,16433,16373,15847,66352,64892,62632,21350,22750,23450,...,76148,74491,71808,"[5200108011, Melilla sección 08011]",5200108011,52001,10314,9961,36128,35196
55083,5200108012 Melilla sección 08012,17350,17185,16792,50730,50458,50839,23450,23450,22050,...,58213,57680,58372,"[5200108012, Melilla sección 08012]",5200108012,52001,10314,9961,36128,35196
55084,5200108013 Melilla sección 08013,12553,12705,11823,37816,38720,36729,16450,16450,15750,...,42633,43654,41356,"[5200108013, Melilla sección 08013]",5200108013,52001,10314,9961,36128,35196
55085,5200108014 Melilla sección 08014,8906,8872,8937,29898,30421,31384,12950,13650,13650,...,32401,33013,34083,"[5200108014, Melilla sección 08014]",5200108014,52001,10314,9961,36128,35196


Rellenamos una de las columnas mediante el método antes planteado, utilizando un for loop. No es la forma más pythonista, desde luego, pero preferimos hacerlo lentamente y ver qué conseguimos.

Mostramos el resultado para Amurrio, que era el tipo de municipio sin datos en sus secciones.

In [30]:
for ind in range(len(rentas_result['RNM per 2017'])):
  if len(str(rentas_result['RNM per 2017'][ind])) < 3:
    rentas_result['RNM per 2017'][ind] = rentas_result['renta_per_2017'][ind]

In [31]:
rentas_result[4:10]

,Territorio,RNM per 2017,RNM per 2016,RNM per 2015,RNM hog 2017,RNM hog 2016,RNM hog 2015,Med R UC 2017,Med R UC 2016,Med R UC 2015,...,RBM hog 2017,RBM hog 2016,RBM hog 2015,Código,Código1,cod_aux,renta_per_2017,renta_per_2015,renta_hog_2017,renta_hog_2015
4,01002 Amurrio,13862,13691,13800,34411,33936,34421,,,,...,,,,"[01002, Amurrio]",01002,01002,13862,13800,34411,34421
5,0100201 Amurrio distrito 01,13862,,,,,,,,,...,,,,"[0100201, Amurrio distrito 01]",0100201,01002,13862,13800,34411,34421
6,0100201001 Amurrio sección 01001,13862,,,,,,,,,...,,,,"[0100201001, Amurrio sección 01001]",0100201001,01002,13862,13800,34411,34421
7,0100201002 Amurrio sección 01002,13862,,,,,,,,,...,,,,"[0100201002, Amurrio sección 01002]",0100201002,01002,13862,13800,34411,34421
8,0100201003 Amurrio sección 01003,13862,,,,,,,,,...,,,,"[0100201003, Amurrio sección 01003]",0100201003,01002,13862,13800,34411,34421
9,0100201004 Amurrio sección 01004,13862,,,,,,,,,...,,,,"[0100201004, Amurrio sección 01004]",0100201004,01002,13862,13800,34411,34421


Aplicamos el for loop a otras tres columnas. Así rellenamos las secciones con los datos de sus municipios, si no tenían datos. Ponemos la condición que la cadena existente inicialmente tenga una longitud inferior a 3, para librarnos de los espacios y puntos, sin tocar los datos numéricos.

In [32]:
for ind in range(len(rentas_result['RNM per 2015'])):
  if len(str(rentas_result['RNM per 2015'][ind])) < 3:
    rentas_result['RNM per 2015'][ind] = rentas_result['renta_per_2015'][ind]

In [33]:
for ind in range(len(rentas_result['RNM hog 2017'])):
  if len(str(rentas_result['RNM hog 2017'][ind])) < 3:
    rentas_result['RNM hog 2017'][ind] = rentas_result['renta_hog_2017'][ind]

In [34]:
for ind in range(len(rentas_result['RNM hog 2015'])):
  if len(str(rentas_result['RNM hog 2015'][ind])) < 3:
    rentas_result['RNM hog 2015'][ind] = rentas_result['renta_hog_2015'][ind]

In [35]:
rentas_result

,Territorio,RNM per 2017,RNM per 2016,RNM per 2015,RNM hog 2017,RNM hog 2016,RNM hog 2015,Med R UC 2017,Med R UC 2016,Med R UC 2015,...,RBM hog 2017,RBM hog 2016,RBM hog 2015,Código,Código1,cod_aux,renta_per_2017,renta_per_2015,renta_hog_2017,renta_hog_2015
0,01001 Alegría-Dulantzi,,,,,,,,,,...,,,,"[01001, Alegría-Dulantzi]",01001,01001,,,,
1,0100101 Alegría-Dulantzi distrito 01,,,,,,,,,,...,,,,"[0100101, Alegría-Dulantzi distrito 01]",0100101,01001,,,,
2,0100101001 Alegría-Dulantzi sección 01001,,,,,,,,,,...,,,,"[0100101001, Alegría-Dulantzi sección 01001]",0100101001,01001,,,,
3,0100101002 Alegría-Dulantzi sección 01002,,,,,,,,,,...,,,,"[0100101002, Alegría-Dulantzi sección 01002]",0100101002,01001,,,,
4,01002 Amurrio,13862,13691,13800,34411,33936,34421,,,,...,,,,"[01002, Amurrio]",01002,01002,13862,13800,34411,34421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55082,5200108011 Melilla sección 08011,16433,16373,15847,66352,64892,62632,21350,22750,23450,...,76148,74491,71808,"[5200108011, Melilla sección 08011]",5200108011,52001,10314,9961,36128,35196
55083,5200108012 Melilla sección 08012,17350,17185,16792,50730,50458,50839,23450,23450,22050,...,58213,57680,58372,"[5200108012, Melilla sección 08012]",5200108012,52001,10314,9961,36128,35196
55084,5200108013 Melilla sección 08013,12553,12705,11823,37816,38720,36729,16450,16450,15750,...,42633,43654,41356,"[5200108013, Melilla sección 08013]",5200108013,52001,10314,9961,36128,35196
55085,5200108014 Melilla sección 08014,8906,8872,8937,29898,30421,31384,12950,13650,13650,...,32401,33013,34083,"[5200108014, Melilla sección 08014]",5200108014,52001,10314,9961,36128,35196


Ahora vamos con los registros de secciones de los que no hay datos siquiera para sus municipios. Y vemos que hay unos cuantos, más de 4.500. Por otro lado, comprobamos que todos las columnas son objetos, no valores numéricos.

In [36]:
rentas_result['RNM per 2017'].value_counts()

.        4522
14367     194
30210     166
13216     128
3431       28
         ... 
6472        1
6469        1
6467        1
22850       1
16383       1
Name: RNM per 2017, Length: 12055, dtype: int64

In [37]:
rentas_result.dtypes

Territorio        object
RNM per 2017      object
RNM per 2016      object
RNM per 2015      object
RNM hog 2017      object
RNM hog 2016      object
RNM hog 2015      object
Med R UC 2017     object
Med R UC 2016     object
Med R UC 2015     object
RBM per 2017      object
RBM per 2016      object
RBM per 2015      object
RBM hog 2017      object
RBM hog 2016      object
RBM hog 2015      object
Código            object
Código1           object
cod_aux           object
renta_per_2017    object
renta_per_2015    object
renta_hog_2017    object
renta_hog_2015    object
dtype: object

Lo que hacemos ahora es tratar una de las columnas que nos sirva como base para tratar las demás, la 'RNM per 2017'.

Comenzamos sustituyendo los espacios y puntos con ceros.

In [38]:
rentas_result['RNM per 2017'] = rentas_result['RNM per 2017'].replace(' ', 0)
rentas_result['RNM per 2017'] = rentas_result['RNM per 2017'].replace('.', 0)


Pasamos ahora la columna a numérica.

In [39]:
rentas_result['RNM per 2017'] = pd.to_numeric(rentas_result['RNM per 2017'])

Calculamos la media de la columna, excluyendo los registros iguales a cero que acabamos de rellenar. Utilizaremos la media para imponerla que son ahora ceros.

In [40]:
avg = rentas_result[rentas_result['RNM per 2017'] > 1]['RNM per 2017'].mean()

In [41]:
avg

11234.267196866283

Ahora aplicamos una función muy simple, celda(x) que sustituye con la media de la columna en los registros que son cero.

In [43]:
def celda(x):
  if x == 0:
    return avg
  else:
    return x

In [44]:
rentas_result['RNM per 2017'] = rentas_result['RNM per 2017'].apply(celda)

In [45]:
avg

11234.267196866283

Comprobamos que en el municipio de Alegría ya tenemos la la mencionada columna rellenada.

In [46]:
rentas_result

,Territorio,RNM per 2017,RNM per 2016,RNM per 2015,RNM hog 2017,RNM hog 2016,RNM hog 2015,Med R UC 2017,Med R UC 2016,Med R UC 2015,...,RBM hog 2017,RBM hog 2016,RBM hog 2015,Código,Código1,cod_aux,renta_per_2017,renta_per_2015,renta_hog_2017,renta_hog_2015
0,01001 Alegría-Dulantzi,11234.267197,,,,,,,,,...,,,,"[01001, Alegría-Dulantzi]",01001,01001,,,,
1,0100101 Alegría-Dulantzi distrito 01,11234.267197,,,,,,,,,...,,,,"[0100101, Alegría-Dulantzi distrito 01]",0100101,01001,,,,
2,0100101001 Alegría-Dulantzi sección 01001,11234.267197,,,,,,,,,...,,,,"[0100101001, Alegría-Dulantzi sección 01001]",0100101001,01001,,,,
3,0100101002 Alegría-Dulantzi sección 01002,11234.267197,,,,,,,,,...,,,,"[0100101002, Alegría-Dulantzi sección 01002]",0100101002,01001,,,,
4,01002 Amurrio,13862.000000,13691,13800,34411,33936,34421,,,,...,,,,"[01002, Amurrio]",01002,01002,13862,13800,34411,34421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55082,5200108011 Melilla sección 08011,16433.000000,16373,15847,66352,64892,62632,21350,22750,23450,...,76148,74491,71808,"[5200108011, Melilla sección 08011]",5200108011,52001,10314,9961,36128,35196
55083,5200108012 Melilla sección 08012,17350.000000,17185,16792,50730,50458,50839,23450,23450,22050,...,58213,57680,58372,"[5200108012, Melilla sección 08012]",5200108012,52001,10314,9961,36128,35196
55084,5200108013 Melilla sección 08013,12553.000000,12705,11823,37816,38720,36729,16450,16450,15750,...,42633,43654,41356,"[5200108013, Melilla sección 08013]",5200108013,52001,10314,9961,36128,35196
55085,5200108014 Melilla sección 08014,8906.000000,8872,8937,29898,30421,31384,12950,13650,13650,...,32401,33013,34083,"[5200108014, Melilla sección 08014]",5200108014,52001,10314,9961,36128,35196


In [47]:
len(rentas_result['RNM per 2017'])

55087

Aplicamos el mismo proceso a las otras tres columnas que nos interesan. De nuevo, no es muy pythonista e hubiese sido más elegante hacerlo mediante una función, pero preferimos ir paso a paso.

In [49]:
rentas_result['RNM per 2015'] = rentas_result['RNM per 2015'].replace(' ', 0)
rentas_result['RNM per 2015'] = rentas_result['RNM per 2015'].replace('.', 0)
rentas_result['RNM per 2015'] = pd.to_numeric(rentas_result['RNM per 2015'])
avg = rentas_result[rentas_result['RNM per 2015'] > 1]['RNM per 2015'].mean()
rentas_result['RNM per 2015'] = rentas_result['RNM per 2015'].apply(celda)

In [50]:
rentas_result

,Territorio,RNM per 2017,RNM per 2016,RNM per 2015,RNM hog 2017,RNM hog 2016,RNM hog 2015,Med R UC 2017,Med R UC 2016,Med R UC 2015,...,RBM hog 2017,RBM hog 2016,RBM hog 2015,Código,Código1,cod_aux,renta_per_2017,renta_per_2015,renta_hog_2017,renta_hog_2015
0,01001 Alegría-Dulantzi,11234.267197,,10618.182737,,,,,,,...,,,,"[01001, Alegría-Dulantzi]",01001,01001,,,,
1,0100101 Alegría-Dulantzi distrito 01,11234.267197,,10618.182737,,,,,,,...,,,,"[0100101, Alegría-Dulantzi distrito 01]",0100101,01001,,,,
2,0100101001 Alegría-Dulantzi sección 01001,11234.267197,,10618.182737,,,,,,,...,,,,"[0100101001, Alegría-Dulantzi sección 01001]",0100101001,01001,,,,
3,0100101002 Alegría-Dulantzi sección 01002,11234.267197,,10618.182737,,,,,,,...,,,,"[0100101002, Alegría-Dulantzi sección 01002]",0100101002,01001,,,,
4,01002 Amurrio,13862.000000,13691,13800.000000,34411,33936,34421,,,,...,,,,"[01002, Amurrio]",01002,01002,13862,13800,34411,34421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55082,5200108011 Melilla sección 08011,16433.000000,16373,15847.000000,66352,64892,62632,21350,22750,23450,...,76148,74491,71808,"[5200108011, Melilla sección 08011]",5200108011,52001,10314,9961,36128,35196
55083,5200108012 Melilla sección 08012,17350.000000,17185,16792.000000,50730,50458,50839,23450,23450,22050,...,58213,57680,58372,"[5200108012, Melilla sección 08012]",5200108012,52001,10314,9961,36128,35196
55084,5200108013 Melilla sección 08013,12553.000000,12705,11823.000000,37816,38720,36729,16450,16450,15750,...,42633,43654,41356,"[5200108013, Melilla sección 08013]",5200108013,52001,10314,9961,36128,35196
55085,5200108014 Melilla sección 08014,8906.000000,8872,8937.000000,29898,30421,31384,12950,13650,13650,...,32401,33013,34083,"[5200108014, Melilla sección 08014]",5200108014,52001,10314,9961,36128,35196


In [51]:
rentas_result['RNM hog 2015'] = rentas_result['RNM hog 2015'].replace(' ', 0)
rentas_result['RNM hog 2015'] = rentas_result['RNM hog 2015'].replace('.', 0)
rentas_result['RNM hog 2015'] = pd.to_numeric(rentas_result['RNM hog 2015'])
avg = rentas_result[rentas_result['RNM hog 2015'] > 1]['RNM hog 2015'].mean()
rentas_result['RNM hog 2015'] = rentas_result['RNM hog 2015'].apply(celda)

In [52]:
rentas_result

,Territorio,RNM per 2017,RNM per 2016,RNM per 2015,RNM hog 2017,RNM hog 2016,RNM hog 2015,Med R UC 2017,Med R UC 2016,Med R UC 2015,...,RBM hog 2017,RBM hog 2016,RBM hog 2015,Código,Código1,cod_aux,renta_per_2017,renta_per_2015,renta_hog_2017,renta_hog_2015
0,01001 Alegría-Dulantzi,11234.267197,,10618.182737,,,26938.114416,,,,...,,,,"[01001, Alegría-Dulantzi]",01001,01001,,,,
1,0100101 Alegría-Dulantzi distrito 01,11234.267197,,10618.182737,,,26938.114416,,,,...,,,,"[0100101, Alegría-Dulantzi distrito 01]",0100101,01001,,,,
2,0100101001 Alegría-Dulantzi sección 01001,11234.267197,,10618.182737,,,26938.114416,,,,...,,,,"[0100101001, Alegría-Dulantzi sección 01001]",0100101001,01001,,,,
3,0100101002 Alegría-Dulantzi sección 01002,11234.267197,,10618.182737,,,26938.114416,,,,...,,,,"[0100101002, Alegría-Dulantzi sección 01002]",0100101002,01001,,,,
4,01002 Amurrio,13862.000000,13691,13800.000000,34411,33936,34421.000000,,,,...,,,,"[01002, Amurrio]",01002,01002,13862,13800,34411,34421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55082,5200108011 Melilla sección 08011,16433.000000,16373,15847.000000,66352,64892,62632.000000,21350,22750,23450,...,76148,74491,71808,"[5200108011, Melilla sección 08011]",5200108011,52001,10314,9961,36128,35196
55083,5200108012 Melilla sección 08012,17350.000000,17185,16792.000000,50730,50458,50839.000000,23450,23450,22050,...,58213,57680,58372,"[5200108012, Melilla sección 08012]",5200108012,52001,10314,9961,36128,35196
55084,5200108013 Melilla sección 08013,12553.000000,12705,11823.000000,37816,38720,36729.000000,16450,16450,15750,...,42633,43654,41356,"[5200108013, Melilla sección 08013]",5200108013,52001,10314,9961,36128,35196
55085,5200108014 Melilla sección 08014,8906.000000,8872,8937.000000,29898,30421,31384.000000,12950,13650,13650,...,32401,33013,34083,"[5200108014, Melilla sección 08014]",5200108014,52001,10314,9961,36128,35196


In [53]:
rentas_result['RNM hog 2017'] = rentas_result['RNM hog 2017'].replace(' ', 0)
rentas_result['RNM hog 2017'] = rentas_result['RNM hog 2017'].replace('.', 0)
rentas_result['RNM hog 2017'] = pd.to_numeric(rentas_result['RNM hog 2017'])
avg = rentas_result[rentas_result['RNM hog 2017'] > 1]['RNM hog 2017'].mean()
rentas_result['RNM hog 2017'] = rentas_result['RNM hog 2017'].apply(celda)

In [54]:
rentas_result

,Territorio,RNM per 2017,RNM per 2016,RNM per 2015,RNM hog 2017,RNM hog 2016,RNM hog 2015,Med R UC 2017,Med R UC 2016,Med R UC 2015,...,RBM hog 2017,RBM hog 2016,RBM hog 2015,Código,Código1,cod_aux,renta_per_2017,renta_per_2015,renta_hog_2017,renta_hog_2015
0,01001 Alegría-Dulantzi,11234.267197,,10618.182737,28322.021999,,26938.114416,,,,...,,,,"[01001, Alegría-Dulantzi]",01001,01001,,,,
1,0100101 Alegría-Dulantzi distrito 01,11234.267197,,10618.182737,28322.021999,,26938.114416,,,,...,,,,"[0100101, Alegría-Dulantzi distrito 01]",0100101,01001,,,,
2,0100101001 Alegría-Dulantzi sección 01001,11234.267197,,10618.182737,28322.021999,,26938.114416,,,,...,,,,"[0100101001, Alegría-Dulantzi sección 01001]",0100101001,01001,,,,
3,0100101002 Alegría-Dulantzi sección 01002,11234.267197,,10618.182737,28322.021999,,26938.114416,,,,...,,,,"[0100101002, Alegría-Dulantzi sección 01002]",0100101002,01001,,,,
4,01002 Amurrio,13862.000000,13691,13800.000000,34411.000000,33936,34421.000000,,,,...,,,,"[01002, Amurrio]",01002,01002,13862,13800,34411,34421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55082,5200108011 Melilla sección 08011,16433.000000,16373,15847.000000,66352.000000,64892,62632.000000,21350,22750,23450,...,76148,74491,71808,"[5200108011, Melilla sección 08011]",5200108011,52001,10314,9961,36128,35196
55083,5200108012 Melilla sección 08012,17350.000000,17185,16792.000000,50730.000000,50458,50839.000000,23450,23450,22050,...,58213,57680,58372,"[5200108012, Melilla sección 08012]",5200108012,52001,10314,9961,36128,35196
55084,5200108013 Melilla sección 08013,12553.000000,12705,11823.000000,37816.000000,38720,36729.000000,16450,16450,15750,...,42633,43654,41356,"[5200108013, Melilla sección 08013]",5200108013,52001,10314,9961,36128,35196
55085,5200108014 Melilla sección 08014,8906.000000,8872,8937.000000,29898.000000,30421,31384.000000,12950,13650,13650,...,32401,33013,34083,"[5200108014, Melilla sección 08014]",5200108014,52001,10314,9961,36128,35196


Ahora con las columnas rellenas, nos quedamos solo con los registros de las secciones, que identificamos porque su código contiene 10 caracteres. El número de registros que obtenemos, más de 36 mil, sabemos que coincide con el de secciones.

In [55]:
rentas_result_sec = rentas_result.loc[rentas_result['Código1'].str.len() == 10]

In [56]:
rentas_result_sec

,Territorio,RNM per 2017,RNM per 2016,RNM per 2015,RNM hog 2017,RNM hog 2016,RNM hog 2015,Med R UC 2017,Med R UC 2016,Med R UC 2015,...,RBM hog 2017,RBM hog 2016,RBM hog 2015,Código,Código1,cod_aux,renta_per_2017,renta_per_2015,renta_hog_2017,renta_hog_2015
2,0100101001 Alegría-Dulantzi sección 01001,11234.267197,,10618.182737,28322.021999,,26938.114416,,,,...,,,,"[0100101001, Alegría-Dulantzi sección 01001]",0100101001,01001,,,,
3,0100101002 Alegría-Dulantzi sección 01002,11234.267197,,10618.182737,28322.021999,,26938.114416,,,,...,,,,"[0100101002, Alegría-Dulantzi sección 01002]",0100101002,01001,,,,
6,0100201001 Amurrio sección 01001,13862.000000,,13800.000000,34411.000000,,34421.000000,,,,...,,,,"[0100201001, Amurrio sección 01001]",0100201001,01002,13862,13800,34411,34421
7,0100201002 Amurrio sección 01002,13862.000000,,13800.000000,34411.000000,,34421.000000,,,,...,,,,"[0100201002, Amurrio sección 01002]",0100201002,01002,13862,13800,34411,34421
8,0100201003 Amurrio sección 01003,13862.000000,,13800.000000,34411.000000,,34421.000000,,,,...,,,,"[0100201003, Amurrio sección 01003]",0100201003,01002,13862,13800,34411,34421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55082,5200108011 Melilla sección 08011,16433.000000,16373,15847.000000,66352.000000,64892,62632.000000,21350,22750,23450,...,76148,74491,71808,"[5200108011, Melilla sección 08011]",5200108011,52001,10314,9961,36128,35196
55083,5200108012 Melilla sección 08012,17350.000000,17185,16792.000000,50730.000000,50458,50839.000000,23450,23450,22050,...,58213,57680,58372,"[5200108012, Melilla sección 08012]",5200108012,52001,10314,9961,36128,35196
55084,5200108013 Melilla sección 08013,12553.000000,12705,11823.000000,37816.000000,38720,36729.000000,16450,16450,15750,...,42633,43654,41356,"[5200108013, Melilla sección 08013]",5200108013,52001,10314,9961,36128,35196
55085,5200108014 Melilla sección 08014,8906.000000,8872,8937.000000,29898.000000,30421,31384.000000,12950,13650,13650,...,32401,33013,34083,"[5200108014, Melilla sección 08014]",5200108014,52001,10314,9961,36128,35196


Nos quedamos en el df de las secciones solo con las columnas que nos interesan, cambiando el nombre de la columna donde viene la sección a cod sec, que utilizaremos cuando hagamos la unificación de los datos. 

In [57]:
rentas_result_sec = rentas_result_sec.rename(columns = {'Código1': 'cod_sec'})

In [58]:
rentas_result_sec.columns

Index(['Territorio', 'RNM per 2017', 'RNM per 2016', 'RNM per 2015',
       'RNM hog 2017', 'RNM hog 2016', 'RNM hog 2015', 'Med R UC 2017',
       'Med R UC 2016', 'Med R UC 2015', 'RBM per 2017', 'RBM per 2016',
       'RBM per 2015', 'RBM hog 2017', 'RBM hog 2016', 'RBM hog 2015',
       'Código', 'cod_sec', 'cod_aux', 'renta_per_2017', 'renta_per_2015',
       'renta_hog_2017', 'renta_hog_2015'],
      dtype='object')

In [59]:
rentas_result_sec = rentas_result_sec[['Territorio', 'cod_sec', 'RNM per 2017', 'RNM per 2015',
       'RNM hog 2017', 'RNM hog 2015']]

In [60]:
rentas_result_sec

,Territorio,cod_sec,RNM per 2017,RNM per 2015,RNM hog 2017,RNM hog 2015
2,0100101001 Alegría-Dulantzi sección 01001,0100101001,11234.267197,10618.182737,28322.021999,26938.114416
3,0100101002 Alegría-Dulantzi sección 01002,0100101002,11234.267197,10618.182737,28322.021999,26938.114416
6,0100201001 Amurrio sección 01001,0100201001,13862.000000,13800.000000,34411.000000,34421.000000
7,0100201002 Amurrio sección 01002,0100201002,13862.000000,13800.000000,34411.000000,34421.000000
8,0100201003 Amurrio sección 01003,0100201003,13862.000000,13800.000000,34411.000000,34421.000000
...,...,...,...,...,...,...
55082,5200108011 Melilla sección 08011,5200108011,16433.000000,15847.000000,66352.000000,62632.000000
55083,5200108012 Melilla sección 08012,5200108012,17350.000000,16792.000000,50730.000000,50839.000000
55084,5200108013 Melilla sección 08013,5200108013,12553.000000,11823.000000,37816.000000,36729.000000
55085,5200108014 Melilla sección 08014,5200108014,8906.000000,8937.000000,29898.000000,31384.000000


Finalmente guardamos el dafaframe de las secciones. En esta ocasión estos datos no dependen de ninguna elección, por lo que este es el único fichero.

In [61]:
rentas_result_sec.to_csv('rentas_secciones_15_17.txt', sep = ',', index = False)

No utilizarmos el dataframe con los datos de los municipios, pero lo guardamos porque podría sernos útil algún día. El número de registros, unos 8.100 coincide con en número real de municipios.

In [62]:
rentas_muni

,cod_aux,renta_per_2017,renta_per_2015,renta_hog_2017,renta_hog_2015
0,01001,,,,
4,01002,13862,13800,34411,34421
13,01003,16177,15786,43021,43199
16,01004,12530,12722,31490,32269
19,01006,13115,13040,31165,30699
...,...,...,...,...,...
54961,50901,16414,16613,25367,26506
54964,50902,.,.,.,.
54967,50903,13018,12136,34237,32498
54971,51001,10767,10696,35274,35577


In [63]:
rentas_muni = rentas_muni.rename(columns = {'cod_aux': 'cod_mun'})

In [64]:
rentas_muni

,cod_mun,renta_per_2017,renta_per_2015,renta_hog_2017,renta_hog_2015
0,01001,,,,
4,01002,13862,13800,34411,34421
13,01003,16177,15786,43021,43199
16,01004,12530,12722,31490,32269
19,01006,13115,13040,31165,30699
...,...,...,...,...,...
54961,50901,16414,16613,25367,26506
54964,50902,.,.,.,.
54967,50903,13018,12136,34237,32498
54971,51001,10767,10696,35274,35577


In [65]:
rentas_muni.to_csv('rentas_municipios_15_17.txt', sep = ',', index = False)

In [66]:
#para guardar el archivo en s3:

from botocore.exceptions import ClientError

s3_client = boto3.client(
    's3',
    aws_access_key_id='XXXXX',
    aws_secret_access_key='XXXXXXXXX',    
)

def upload_file(file_name, bucket, object_name=None):
    """Subir un archivo a un bucket
    :param file_name: archivo que hay que subir
    :param bucket: Bucket al que hay que subirlo
    :param object_name: S3 object name. Incluye la carpeta en la que hay que guardarlo. si no hay no se pone nada
    :return: True si sube el archivo, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    #s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [67]:
upload_file('rentas_municipios_15_17.txt',
            'electomedia',
            object_name = "INE/" + 'rentas_municipios_15_17.txt'
           )

True

In [68]:
upload_file('rentas_secciones_15_17.txt',
            'electomedia',
            object_name = "INE/" + 'rentas_secciones_15_17.txt'
           )

True